<a href="https://colab.research.google.com/github/siddharth-kale/Twitter-Network-Analysis/blob/main/Stock_Twitter_Influencers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sea
!pip install snscrape
import snscrape.modules.twitter as sntwitter
import re
!pip install networkx
import networkx as nx

In [2]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [3]:
pd.options.display.max_rows=1000

In [4]:
keyword = '$BIDU'
startdate = "2019-01-01"
enddate = "2021-11-20"

In [5]:
maxtweets = 5000000
tweetlist=[]
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('{} since:{} until:{}'.format(keyword,startdate,enddate)).get_items()):
   if i>maxtweets:
         break
   tweetlist.append(tweet)

tweets_df = pd.DataFrame(tweetlist)

In [6]:
tweets_df.to_csv('/content/gdrive/MyDrive/Projects/Twitter Network Analysis/Data/{}Tweets.csv'.format(keyword))

In [7]:
tweets_df  = pd.read_csv('/content/gdrive/MyDrive/Projects/Twitter Network Analysis/Data/{}Tweets.csv'.format(keyword),low_memory=False,lineterminator='\n')

In [8]:
tweets_df

,Unnamed: 0,url,date,content,id,username,outlinks,outlinksss,tcooutlinks,tcooutlinksss
0,0,https://twitter.com/Trading4JP/status/14618439...,2021-11-19 23:48:47+00:00,Third week of November scan\nOutside week\n$BA...,1461843928743694336,Trading4JP,[],NaN,[],NaN
1,1,https://twitter.com/newsfilterio/status/146184...,2021-11-19 23:41:47+00:00,China’s Transition Is Risky for Investors. Her...,1461842168226668545,newsfilterio,['https://newsfilter.io/articles/chinas-transi...,https://newsfilter.io/articles/chinas-transiti...,['https://t.co/5RHt0kY7Qx'],https://t.co/5RHt0kY7Qx
2,2,https://twitter.com/newsfilterio/status/146183...,2021-11-19 23:29:47+00:00,"Europe lockdown fears rattle investors, spark ...",1461839148160102413,newsfilterio,['https://newsfilter.io/articles/europe-lockdo...,https://newsfilter.io/articles/europe-lockdown...,['https://t.co/CGw4y5yaXG'],https://t.co/CGw4y5yaXG
3,3,https://twitter.com/leodepisa/status/146183874...,2021-11-19 23:28:10+00:00,Top ten PEOR desempeño semanal de papeles que ...,1461838743023673345,leodepisa,[],NaN,[],NaN
4,4,https://twitter.com/VapzerSignals/status/14618...,2021-11-19 22:25:32+00:00,Scanner\n#stocks @ 11/19/2021 2:25 PM PST\nTF:...,1461822978497269766,VapzerSignals,[],NaN,[],NaN
...,...,...,...,...,...,...,...,...,...,...
99783,99783,https://twitter.com/StockflareUS/status/108002...,2019-01-01 09:09:20+00:00,$BIDU: Baidu Inc (ADR)'s forecast earnings mov...,1080028186753355777,StockflareUS,['https://stockflare.com/stock/BIDU.O'],https://stockflare.com/stock/BIDU.O,['https://t.co/Ikm27bFWXG'],https://t.co/Ikm27bFWXG
99784,99784,https://twitter.com/StockflareUS/status/108002...,2019-01-01 09:09:19+00:00,$BIDU: Baidu Inc (ADR)'s price is -15.8% but t...,1080028184840716288,StockflareUS,['https://stockflare.com/stock/BIDU.O'],https://stockflare.com/stock/BIDU.O,['https://t.co/Ikm27bFWXG'],https://t.co/Ikm27bFWXG
99785,99785,https://twitter.com/shortvolumes/status/107997...,2019-01-01 05:36:02+00:00,Short sale vol (not short interest) for $NEAR ...,1079974508616994822,shortvolumes,['http://shortvolumes.com/?t=NEAR'],http://shortvolumes.com/?t=NEAR,['https://t.co/tdsvmCwaTo'],https://t.co/tdsvmCwaTo
99786,99786,https://twitter.com/ExactOptionPick/status/107...,2019-01-01 01:00:27+00:00,251 Profit-Packed Option Trades Yearly--Delive...,1079905157352165376,ExactOptionPick,['http://ow.ly/wnKK30n8ZUl'],http://ow.ly/wnKK30n8ZUl,['https://t.co/ePnqPmgWVF'],https://t.co/ePnqPmgWVF


In [9]:
#Code to filter out only the ticker
templist=[]

for i in range(0,len(tweets_df)):
  
  if keyword[0]=='$':    
    templist=re.findall('\$\w+',str(tweets_df.iloc[i,3]))
  else:
    templist=re.findall('(?i){}'.format(keyword),str(tweets_df.iloc[i,3]))
  
  tweets_df.loc[i,'TickerCount'] = len(templist)
  templist.clear()

onlyticker = tweets_df[tweets_df['TickerCount']==1]
onlyticker = onlyticker.drop(columns=['Unnamed: 0']).reset_index().drop(columns='index')
onlyticker

,url,date,content,id,username,outlinks,outlinksss,tcooutlinks,tcooutlinksss,TickerCount
0,https://twitter.com/VapzerSignals/status/14618...,2021-11-19 22:25:32+00:00,Scanner\n#stocks @ 11/19/2021 2:25 PM PST\nTF:...,1461822978497269766,VapzerSignals,[],NaN,[],NaN,1.0
1,https://twitter.com/dailycandlestix/status/146...,2021-11-19 21:20:58+00:00,$BIDU Minor Breakdown\nDaily appearances since...,1461806731818778624,dailycandlestix,[],NaN,[],NaN,1.0
2,https://twitter.com/TradeWithAlerts/status/146...,2021-11-19 20:33:32+00:00,A nice print for $BIDU at 3:33pm -- SIDE: Near...,1461794795127545857,TradeWithAlerts,['https://tradewithalerts.com/last-print/BIDU'...,https://tradewithalerts.com/last-print/BIDU ht...,"['https://t.co/wOzt2qqlSF', 'https://t.co/VC1L...",https://t.co/wOzt2qqlSF https://t.co/VC1LdPHaJ...,1.0
3,https://twitter.com/NohaWolfe/status/146177109...,2021-11-19 18:59:23+00:00,$BIDU ~Do you want option or stock alerts on t...,1461771099444629507,NohaWolfe,['https://discord.gg/hHyAcn6AFZ'],https://discord.gg/hHyAcn6AFZ,['https://t.co/7faSSkpeEZ'],https://t.co/7faSSkpeEZ,1.0
4,https://twitter.com/pingkind/status/1461767815...,2021-11-19 18:46:20+00:00,$BIDU weekly bear flag broken downward. https:...,1461767815061336066,pingkind,[],NaN,[],NaN,1.0
...,...,...,...,...,...,...,...,...,...,...
25853,https://twitter.com/CFinancialNews/status/1080...,2019-01-01 13:00:36+00:00,Baidu Inc $BIDU Receives Consensus Recommendat...,1080086386504617984,CFinancialNews,['http://americanbankingnews.com/?p=9126354'],http://americanbankingnews.com/?p=9126354,['https://t.co/Ll4n1d8BJZ'],https://t.co/Ll4n1d8BJZ,1.0
25854,https://twitter.com/AmericanBanking/status/108...,2019-01-01 13:00:35+00:00,Baidu Inc $BIDU Receives Consensus Recommendat...,1080086381458804736,AmericanBanking,['http://americanbankingnews.com/?p=9126354'],http://americanbankingnews.com/?p=9126354,['https://t.co/03y3AfxSFJ'],https://t.co/03y3AfxSFJ,1.0
25855,https://twitter.com/ThisLincolnian/status/1080...,2019-01-01 12:59:31+00:00,Baidu Inc $BIDU Receives Consensus Recommendat...,1080086115560947712,ThisLincolnian,['http://thelincolnianonline.com/?p=2598942'],http://thelincolnianonline.com/?p=2598942,['https://t.co/f4s9LpkNa5'],https://t.co/f4s9LpkNa5,1.0
25856,https://twitter.com/StockflareUS/status/108002...,2019-01-01 09:09:20+00:00,$BIDU: Baidu Inc (ADR)'s forecast earnings mov...,1080028186753355777,StockflareUS,['https://stockflare.com/stock/BIDU.O'],https://stockflare.com/stock/BIDU.O,['https://t.co/Ikm27bFWXG'],https://t.co/Ikm27bFWXG,1.0


In [10]:
interaction_df = pd.DataFrame(columns=['source','target'])
ind=0
for i in range(0,len(onlyticker)):
  templist=[]
  templist=re.findall('@\w+',str(onlyticker.iloc[i,2]))
  
  if len(templist)==0:
    interaction_df.loc[ind,'source'] = onlyticker.iloc[i,4]
    interaction_df.loc[ind,'target'] = ''
    ind = ind+1

  else:
    for j in range(0,len(templist)):
      interaction_df.loc[ind,'source'] = onlyticker.iloc[i,4]
      interaction_df.loc[ind,'target'] = templist[j][1:]
      ind = ind+1

interaction_df



,source,target
0,VapzerSignals,
1,dailycandlestix,
2,TradeWithAlerts,
3,NohaWolfe,
4,pingkind,
...,...,...
26815,CFinancialNews,
26816,AmericanBanking,
26817,ThisLincolnian,
26818,StockflareUS,


In [11]:
interaction_df = interaction_df[interaction_df['target'].isin(interaction_df['source'])]

In [12]:
filterout=[]

In [13]:
nodes_df = pd.DataFrame(interaction_df.stack()).reset_index().drop(columns=['level_0','level_1'])
nodes_df.columns=['username']
complete_nodes=list(set(nodes_df[~nodes_df['username'].isin(filterout)]['username'].to_list()))

In [14]:
G = nx.DiGraph()

In [15]:
for i in range(0,len(complete_nodes)):
  G.add_node(complete_nodes[i])

print(G.number_of_nodes())

1576


In [16]:
full_interaction_df = interaction_df[~interaction_df['target'].isin(filterout)]
for i in range(0,len(full_interaction_df)):
  G.add_edge(full_interaction_df.iloc[i,0],full_interaction_df.iloc[i,1])

In [17]:
nodedegree=pd.DataFrame.from_dict(dict(G.in_degree),orient='index')
nodedegree = nodedegree.reset_index()
nodedegree.columns=['Name','InDegree']
nodedegree = nodedegree.sort_values(by='InDegree',ascending=False)
nodedegree.head(10)

,Name,InDegree
573,TrendSpider,42
1246,traderstewie,35
361,data168,33
878,Baidu_Inc,28
1323,StockDweebs,26
172,ripster47,24
198,StockMKTNewz,22
254,saxena_puru,20
844,EliteOptions2,19
388,skaushi,14


In [18]:
d =dict(G.in_degree)
plt.figure(figsize=(400,400))
plt.title('Twitter Interaction Graph for {}'.format(keyword))
nx.draw(G,node_size=[(x+1) * 100 for x in list(d.values())],with_labels=True)
plt.savefig('{}.png'.format(keyword))